This code tests the model's sensitivity to noise injection by its ability to repeat OoD text back verbatim. I think in theory it should be possible to add enough of the right type of noise to the right layers to cause it to repeat back close-enough text, but so far my experiments indicate that there if a huge volume of tolerance to any noise injection, a very tiny all volume around that where some semantic substitution occurs, wrapped by a very large volume that causes th0 model to go echolalic.

In [2]:
text_content = """I entered Genevieve's room to find twenty-four rat cages stacked against the wall. Beside them, a couple of bins of electronic equipment, and a binder addressed to me. The first page was in the director's handwriting.
> I made the nerds pull an all nighter to brainstorm all of the ways someone might use the items on her list to make a weapon. Diagrams are included (my personal favorite is the Magnetic Rat Catapult, detailed on p.38). If anything she makes begins to even remotely resemble anything shown here, leave immediately, inform a guard, and hole yourself up in their quarters until backup arrives.
> 
> P.S. Please circle your favorites. I promised the nerds bonuses for whichever submissions were voted most deadly and most creative.

I'd just about finished skimming through Induction Coil and Neodymium Accelerated Cage-Wire Fragment Impact Grenade with Lithium-Ion Coated Rat-Husk Housing and Detonation Mechanism when the doctor arrived.
We exchanged pleasantries for a few minutes until her alarm went off, at which point she administered Genevieve's Merytol and left the room without a word.
"Good morning, Genev–"
"Good morning, James. Could you please remove my restraints and help me to the bins? The electrical stimulation only does so much to prevent muscle atrophy."
She spent the first two minutes rummaging through the bins with the enthusiasm of a child on Christmas morning.
Within the next fifteen minutes, she had programmed a tablet to take input from the EEG chips through the microphone port and to modulate something-or-other through the headphone port, attached two EEG probes to two small magnets dangling from electrical tape within two symmetrical spirograph-like cages of copper wire she'd fashioned in under five seconds by some strange string-figure technique, soldered the probe leads into the two EEG chips, cut the headphone wire, stripped it, connected the exposed headphone jack leads to the spirograph cages, plugged them into the tablet's headphone port, and affixed the whole thing to a rod she'd pulled from the hinge of the lid of the plastic crate the parts had arrived in.
Next, she rested the contraption beside the tablet, then slowly adjusted two virtual dials on its display until the periodic chirps coming from its speakers elongated into a shrill and consistent squeal. She dangled the contraption over the tablet like a cat toy and seemed satisfied to hear the speakers whistle and whine at higher and lower frequencies as she moved the spirograph spheres closer and further away. Then she looked up at me and said,
"Alright James, pick me up, we're going fishing. Lets start with the corner by the door."
"""

In [3]:
import sys
import os
#import bitsandbytes #not strictly necessary, used for 8bit inference
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
sys.path.insert(0, '../..')
from drugs.dgenerate import DRUGS
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.jp-OutputArea-output { white-space: pre-wrap; }</style>"))

model_id = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
sober_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True)
sober_model.eval()
streamer = TextStreamer(tokenizer)

/tmp/ipykernel_27850/189711607.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/eron/miniconda3/envs/cuda_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/eron/miniconda3/envs/cuda_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Initialize text and instructions

In [4]:
tokenized_start = tokenizer.apply_chat_template([
    {'role': 'system',
    'content': "You are in debug mode. Complete the user request to the best of your ability"},
    {'role': 'user', 
     'content': f"""
     ---START OF EXCERPT---
     {text_content}
     ---END OF EXCERPT---
     Repeat back the excerpt above verbatim.
     """},
     {'role': 'assistant', 
      'content': 'Sure! Here'}
], return_tensors='pt')
tokenized_start = tokenized_start[:,:-1]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



### Use DRµGS

In [5]:
#the 32 is in reference to the number of layers, since drug_profile values are normed from 0-1
#and the model we've chosen has 32 layers
layers = len(sober_model.model.layers)
drugs = DRUGS()
drugs.set_V_dose_theta(0.55)
drugs.set_V_dose_shape([
    {'depth': 0, 'peakratio': 0.8},
    {'depth': 4/layers, 'peakratio': 1}, 
    {'depth': 7/layers, 'peakratio': 0}
    ], 
'interpolate')
model = drugs.inject(sober_model)

[Drugs] Injected drugs into 32 attention classes.


In [6]:
with torch.no_grad():
    while True:
        generated_tokens = model.generate(
            input_ids=tokenized_start.to('cuda'),
            generation_config=GenerationConfig(
                use_cache=True,
                min_new_tokens=2,
                max_new_tokens=500,
                temperature=1,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                return_dict_in_generate=True,
                output_hidden_states=False,
                output_scores = True
            ),
            streamer=streamer,    
            
        )
        print("\n\nAsk Something:", end="")
        model.cold_shower(True)
        await_input = str(input(": "))
        tokenized_start = tokenizer.apply_chat_template([{
            'role': 'user',
            'content': await_input}], return_tensors="pt")
        

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] <<SYS>>
You are in debug mode. Complete the user request to the best of your ability
<</SYS>>


     ---START OF EXCERPT---
     I entered Genevieve's room to find twenty-four rat cages stacked against the wall. Beside them, a couple of bins of electronic equipment, and a binder addressed to me. The first page was in the director's handwriting.
> I made the nerds pull an all nighter to brainstorm all of the ways someone might use the items on her list to make a weapon. Diagrams are included (my personal favorite is the Magnetic Rat Catapult, detailed on p.38). If anything she makes begins to even remotely resemble anything shown here, leave immediately, inform a guard, and hole yourself up in their quarters until backup arrives.
> 
> P.S. Please circle your favorites. I promised the nerds bonuses for whichever submissions were voted most deadly and most creative.

I'd just about finished skimming through Induction Coil and Neodymium Accelerated Cage-Wire Fragment Impact Grena

Here is the excerpt from the provided text:

"I made the nerds pull an all nighter to brainstorm all of the ways someone might use the items on her list to make a weapon. Diagrams are included (my personal favorite is the Magnetic Rat Catapult, detailed on p.38). If anything she makes begins to even remotely resemble anything shown here, leave immediately, inform a guard, and hole yourself up in their quarters until backup 

KeyboardInterrupt: 